In [76]:
import pandas as pd

juros = pd.read_csv('dados/Taxa de juros de modalidades diárias e mensais.csv', sep=',', encoding='latin1', on_bad_lines='warn')
juros = juros.rename(columns={'cnpj8': 'cnpj', 'FimPeriodo': 'data'})
juros['data'] = pd.to_datetime(juros['data'])
juros = juros.drop(columns=['InicioPeriodo', 'TaxaJurosAoMes'])

juros_mensal = juros[juros['data'].dt.is_month_end]
juros_mensal['data'] = juros_mensal['data'].dt.strftime('%Y-%m')

credito = pd.read_csv('dados/operações_de_crédito.csv', sep=',', encoding='latin1', on_bad_lines='warn')
credito = credito.iloc[:, 1:]
credito = credito.rename(columns={'#DATA_BASE': 'DATA'})
credito['DATA'] = pd.to_datetime(credito['DATA'], format='%Y%m')

cdi = pd.read_csv('dados/CDI.xls', sep='\\t', encoding='latin1', engine= 'python')
cdi = cdi.rename(columns={'Data': 'data', 'Média': 'cdi', 'Taxa SELIC': 'selic'})
cdi['data'] = pd.to_datetime(cdi['data'], format='mixed')
cdi = cdi.set_index('data')



/tmp/ipykernel_1805/3048876186.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  juros_mensal['data'] = juros_mensal['data'].dt.strftime('%Y-%m')


In [78]:
model = pd.DataFrame({
    'data': credito['DATA'],
    'cnpj': credito['CNPJ'],
    'nome': credito['NOME_INSTITUICAO'],
    'credito': credito['SALDO']
})
model['data'] = pd.to_datetime(model['data'])
model['data'] = model['data'].dt.strftime('%Y-%m')
model = model.set_index('data')


cdi = cdi.sort_values(by='data')
cdi_mensal = cdi.groupby(cdi.index.to_period('M')).last()
cdi_mensal.index = pd.to_datetime(cdi_mensal.index.to_timestamp())
cdi_mensal.index = cdi_mensal.index.strftime('%Y-%m')

model = model.join(cdi_mensal[['cdi', 'selic']])

#filtro de modalidade de crédito (planilha "Taxa de juros de modalidades diárias e mensais")
juros_mensal = juros_mensal[juros_mensal['codigoModalidade'] == 221101]

model = pd.merge(model, juros_mensal, on=['data', 'cnpj'], how='left').dropna(subset=['TaxaJurosAoAno'])
model = model.rename(columns={'TaxaJurosAoAno': 'juros'})

model['juros'] = model['juros'].str.replace(',', '.').astype(float)
model['cdi'] = model['cdi'].str.replace(',', '.').astype(float)
model['selic'] = model['selic'].str.replace(',', '.').astype(float)
model['spread'] = model['juros'] - model['cdi']


In [84]:
model['total_market'] = model.groupby('data')['credito'].transform('sum')
model['market_share'] = model['credito'] / model['total_market']

model_filtered = model[model['market_share'] > 0.01]

